In [2]:
import pandas as pd

In [3]:
schools_data = pd.read_csv("Resources/schools_complete.csv")
students_data = pd.read_csv("Resources/students_complete.csv")

In [4]:
complete_data = pd.merge(schools_data, students_data, on = ["school_name","school_name"])
complete_data.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [5]:
# calculate the total number of schools
total_schools = complete_data["school_name"].nunique()

# calculate the total number of students
total_students = complete_data["student_name"].count()

# calculate the total budget
budget_list = complete_data.groupby(["school_name"])
budget = budget_list["budget"].mean()
total_budget = "${:,.2f}".format(budget.sum())

In [6]:
# calculate the average math score
average_math = complete_data["math_score"].mean()

# calculate the average reading score
average_reading = complete_data["reading_score"].mean()

In [7]:
# calculate the percentage of students with a passing math score (>=70)
passed_m = complete_data.loc[complete_data["math_score"]>=70,:]
math_pass_percentage = passed_m["math_score"].count()/complete_data["math_score"].count()*100

# calculate the percentage of students with a passing reading score (>=70)
passed_r = complete_data.loc[complete_data["reading_score"]>=70,:]
reading_pass_percentage = passed_r["reading_score"].count()/complete_data["reading_score"].count()*100

# calculate the percentage of overall passing
passed_o = complete_data.loc[(complete_data["math_score"]>=70) & (complete_data["reading_score"]>=70),:]
overall_pass_percetage = passed_o["student_name"].count()/complete_data["student_name"].count()*100

In [8]:
# create a dataframe to show the results
results = {"Total Schools" : total_schools,
           "Total Students" : total_students,
           "Total Budget" : total_budget,
           "Average Math Score" : average_math,
           "Average Reading Score" : average_reading,
           "% Passing Math" : math_pass_percentage,
           "% Passing Reading" : reading_pass_percentage,
           "% Overall Passing" : overall_pass_percetage
           }
results_df = pd.DataFrame(results, index=[0])
results_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [9]:
# create an overview table that summarizes key metrics about each school

schools_table = complete_data[["school_name","type","budget","size"]].drop_duplicates()
per_student_budget = pd.DataFrame({"Per Student Budget" : (schools_table["budget"]/schools_table["size"]),
                                   "school_name" : schools_table["school_name"]})
schools_table ["Per Student Budget"] = (schools_table["budget"]/schools_table["size"]).map("${:.2f}".format)
average_scores = pd.DataFrame(complete_data.groupby("school_name")["math_score","reading_score"].mean())
schools_table = pd.merge(schools_table, average_scores, on="school_name")
pass_number = pd.merge(passed_o.groupby("school_name")["student_name"].count(),pd.merge(passed_m.groupby("school_name")["math_score"].count(),passed_r.groupby("school_name")["reading_score"].count(),on="school_name"),on="school_name")
pass_number = pd.merge(pass_number, schools_table[["school_name","size"]], on=["school_name"])
pass_number["% Passing Math"] = pass_number["math_score"]/pass_number["size"]*100
pass_number["% Passing Reading"] = pass_number["reading_score"]/pass_number["size"]*100
pass_number["% Overall Passing"] = pass_number["student_name"]/pass_number["size"]*100
schools_table = pd.merge(schools_table, pass_number[["school_name","% Passing Math","% Passing Reading","% Overall Passing"]], on="school_name")
schools_table = schools_table.rename(columns={
                                     "type" : "School Type",
                                     "budget" : "Total School Budget",
                                     "math_score" : "Average Math Score",
                                     "reading_score" : "Average Reading Score",
                                     "size" : "Total Students"
                                    })
schools_table["Total School Budget"] = schools_table["Total School Budget"].map("${:,.2f}".format)
schools_table = schools_table.set_index("school_name")  
schools_table

C:\Users\terme\AppData\Local\Temp\ipykernel_22504\1298500544.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_scores = pd.DataFrame(complete_data.groupby("school_name")["math_score","reading_score"].mean())


,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Huang High School,District,"$1,910,635.00",2917,$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,"$1,056,600.00",1761,$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,"$3,022,020.00",4635,$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,"$917,500.00",1468,$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"$1,319,574.00",2283,$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,"$3,124,928.00",4976,$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,"$248,087.00",427,$581.00,83.803279,83.814988,92.505855,96.252927,89.227166


In [10]:
# display the top 5 performing schools by overall passing percentage
schools_table.sort_values(by="% Overall Passing", ascending=False).head(5)

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"$1,043,130.00",1635,$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"$917,500.00",1468,$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"$1,319,574.00",2283,$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,"$585,858.00",962,$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [11]:
# display the 5 worst-performing schools by overall passing percentage
schools_table.sort_values(by="% Overall Passing").head(5)

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"$2,547,363.00",3999,$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"$1,910,635.00",2917,$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"$3,022,020.00",4635,$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"$3,094,650.00",4761,$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [12]:
# create a table that lists the average Math Score for students of each grade level
grade_9th_m = pd.DataFrame(complete_data.loc[complete_data["grade"]=="9th",:].groupby("school_name")["math_score"].mean())
grade_10th_m = pd.DataFrame(complete_data.loc[complete_data["grade"]=="10th",:].groupby("school_name")["math_score"].mean())
grade_11th_m = pd.DataFrame(complete_data.loc[complete_data["grade"]=="11th",:].groupby("school_name")["math_score"].mean())
grade_12th_m = pd.DataFrame(complete_data.loc[complete_data["grade"]=="12th",:].groupby("school_name")["math_score"].mean())
math_by_grade = pd.merge(pd.merge(pd.merge(grade_9th_m,grade_10th_m, on=["school_name"], suffixes=["_9th","_10th"]),grade_11th_m, on=["school_name"]),grade_12th_m, on=["school_name"])
math_by_grade = math_by_grade.rename(columns={"math_score_9th" : "9th",
                                              "math_score_10th" : "10th",
                                              "math_score_x" : "11th",
                                              "math_score_y" : "12th"
                                             })
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [13]:
# create a table that lists the average Reading Score for students of each grade level
grade_9th_r = pd.DataFrame(complete_data.loc[complete_data["grade"]=="9th",:].groupby("school_name")["reading_score"].mean())
grade_10th_r = pd.DataFrame(complete_data.loc[complete_data["grade"]=="10th",:].groupby("school_name")["reading_score"].mean())
grade_11th_r = pd.DataFrame(complete_data.loc[complete_data["grade"]=="11th",:].groupby("school_name")["reading_score"].mean())
grade_12th_r = pd.DataFrame(complete_data.loc[complete_data["grade"]=="12th",:].groupby("school_name")["reading_score"].mean())
reading_by_grade = pd.merge(pd.merge(pd.merge(grade_9th_r,grade_10th_r, on=["school_name"], suffixes=["_9th","_10th"]),grade_11th_r, on=["school_name"]),grade_12th_r, on=["school_name"])
reading_by_grade = reading_by_grade.rename(columns={"reading_score_9th" : "9th",
                                              "reading_score_10th" : "10th",
                                              "reading_score_x" : "11th",
                                              "reading_score_y" : "12th"
                                             })
reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [14]:
performance_table = pd.merge(per_student_budget, schools_table[["Total Students","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]], on="school_name")
del performance_table["school_name"]

In [15]:
# create a table that breaks down school performance based on average Spending Ranges
bins = [0,585,630,645,680]
group_labels = ["<$585","$585-630","$630-645","$645-680"]
performance_table["Spending Ranges (Per Student)"] = pd.cut(performance_table["Per Student Budget"], bins, labels=group_labels)
spending_perf = performance_table.groupby("Spending Ranges (Per Student)").mean().round(decimals=2)
del spending_perf["Total Students"]
del spending_perf["Per Student Budget"]
spending_perf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [16]:
# create a table that breaks down school performance based on average School Size
bins1 = [0,1000,2000,5000]
group_labels1 = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]
performance_table["School Size"] = pd.cut(performance_table["Total Students"], bins1, labels=group_labels1)
size_perf = performance_table.groupby("School Size").mean()
del size_perf["Per Student Budget"]
del size_perf["Total Students"]
size_perf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [17]:
# create a table that breaks down school performance based on average School Type
type_perf = schools_table.groupby("School Type").mean()
del type_perf["Total Students"]
type_perf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


# Summary of Analysis
This data is desribing 15 schools with 39170 students, containing details about the score of students in math and reading class.
In this challenge we categorized schools by different metrics such as their type, size and budget and evaluate scores of students for math, reading, overall,and the passing percentage to analyze the performance of each school.
One interesting insight which is obtained by looking at the overall passing percentage, is that all of the top performing schools are Charter schools, also smaller schools with a total budget around 1 mollion dollars, and all of the worst-performing schools are District, larger and also with a budget almost doubled!!
If we take a closer look to average scores by grade level table, we can understand that there's not much of a noticable difference between different grade levels of each school and the tolerance is mainly around 1%.
The table with spending ranges demonstrates that the relation between budget for each student and overall passing percentage is inverse, meaning the higher the budget is, the lower the performance and the average scores are.
We can also see that when the school size is larger than 2000 students the overall prformance, average scores and number of passing students has a considerable decrease.
Finally, comparing schools only by their type, we can understand that average math score and number of student that passed the math class are much lower in district schools than charter schools, this difference in reading score averages is less but still charter schools have a better overall performance.  
